In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
# 데이터 경로 설정
base_path = 'resized_feature'  # 경로를 실제 Google Drive에 있는 skindata 폴더 경로로 변경하세요.

# 얼굴 부위 및 피부 상태 설정
face_parts = ['forehead', 'nose', 'cheek']

# 데이터 불러오기
data = []
labels = []

for part in face_parts:
    folder_path = os.path.join(base_path, 'resized_train', part)
        
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)

        # Additional check to skip empty images
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)

            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (128, 128))

                data.append(img)
                labels.append(f'{part}') 
            else:
                print(f"Warning: Unable to read the image at {img_path}")

# 데이터와 레이블을 numpy 배열로 변환
data = np.array(data)
labels = np.array(labels)

# 레이블 인코딩
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Check if the arrays are not empty
if data.shape[0] > 0 and encoded_labels.size > 0:
    # 레이블 인코딩
    encoded_labels = to_categorical(encoded_labels)

    # 데이터 분할 (8:2 비율)
    X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.2, random_state=42)
else:
    print("Error: No data or labels to split.")


In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128,128,3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
9/9 [==============================] - 36s 3s/step - loss: 111.9330 - accuracy: 0.3663 - val_loss: 1.2975 - val_accuracy: 0.4097
Epoch 2/10
9/9 [==============================] - 28s 3s/step - loss: 0.9236 - accuracy: 0.6580 - val_loss: 0.6722 - val_accuracy: 0.7431
Epoch 3/10
9/9 [==============================] - 23s 3s/step - loss: 0.5526 - accuracy: 0.8247 - val_loss: 0.6895 - val_accuracy: 0.7708
Epoch 4/10
9/9 [==============================] - 18s 2s/step - loss: 0.4600 - accuracy: 0.8333 - val_loss: 1.3128 - val_accuracy: 0.6875
Epoch 5/10
9/9 [==============================] - 20s 2s/step - loss: 0.5152 - accuracy: 0.8229 - val_loss: 0.5164 - val_accuracy: 0.7778
Epoch 6/10
9/9 [==============================] - 19s 2s/step - loss: 0.4040 - accuracy: 0.8333 - val_loss: 0.7724 - val_accuracy: 0.7847
Epoch 7/10
9/9 [==============================] - 18s 2s/step - loss: 0.4010 - accuracy: 0.8507 - val_loss: 0.5521 - val_accuracy: 0.8056
Epoch 8/10
9/9 [================

In [5]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# 테스트 데이터에 대한 예측
predictions = model.predict(X_test)

# 예측 결과 출력
for i in range(5):  # 예측 결과 중에서 처음 5개만 출력
    predicted_class = label_encoder.classes_[np.argmax(predictions[i])]
    actual_class = label_encoder.classes_[np.argmax(y_test[i])]
    print(f'Actual: {actual_class}, Predicted: {predicted_class}')

5/5 [==============================] - 2s 288ms/step - loss: 0.6096 - accuracy: 0.8194
Test Loss: 0.6095917224884033, Test Accuracy: 0.8194444179534912
Actual: nose, Predicted: forehead
Actual: nose, Predicted: nose
Actual: forehead, Predicted: forehead
Actual: forehead, Predicted: forehead
Actual: nose, Predicted: nose
